In [1]:
using Pkg
Pkg.add("IJulia")
Pkg.add("ITensors")
Pkg.add("ITensorMPS")
Pkg.add("LinearAlgebra")
Pkg.add("Random")
Pkg.add("Plots")

┌ Error: curl_easy_setopt: 4
└ @ Downloads.Curl C:\Users\Leonardo.Borello\AppData\Local\Programs\Julia-1.11.4\share\julia\stdlib\v1.11\Downloads\src\Curl\utils.jl:50
   Resolving package versions...
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
 

In [76]:
using ITensors, ITensorMPS, LinearAlgebra

function generate_mps_xyz(
    N::Int,
    Jx::Float64,
    Jy::Float64,
    Jz::Float64,
    h::Float64,
    nsweeps::Int,
    cutoff::Vector{Float64},
)
    sites = siteinds("S=1/2", N)


    os = OpSum()
    for j in 1:N-1
        os += Jx, "Sx", j, "Sx", j+1
        os += Jy, "Sy", j, "Sy", j+1
        os += Jz, "Sz", j, "Sz", j+1
    end
    for j in 1:N
        os += -h, "Sx", j
    end
    H = MPO(os, sites)

    # 3) Initialize a random MPS (here all spins "Dn")
    psi0 = random_mps(sites, n -> "Dn")

    # 4) Run DMRG
    energy, psi1 = dmrg(H, psi0; nsweeps=nsweeps, cutoff=cutoff)

    # 5) Left‐canonicalize
    psi_lcanon = orthogonalize!(psi1, 1)

    return psi_lcanon, energy
end


N       = 8 
Jx      = 5.0         
Jy      = 6.0         
Jz      = 5.0         
h       = 0.5       
nsweeps = 5
cutoff  = [1e-10]

psi_non_truncated, Energy = generate_mps_xyz(N, Jx, Jy, Jz, h, nsweeps, cutoff)
sites = siteinds(psi_non_truncated)
product_state = MPS(sites, "Up")

After sweep 1 energy=-17.84952924762829  maxlinkdim=4 maxerr=9.99E-16 time=0.017
After sweep 2 energy=-18.023248219232176  maxlinkdim=16 maxerr=2.70E-16 time=0.005
After sweep 3 energy=-18.023253348703808  maxlinkdim=16 maxerr=2.05E-16 time=0.006
After sweep 4 energy=-18.023253348705484  maxlinkdim=16 maxerr=4.40E-16 time=0.006
After sweep 5 energy=-18.023253348705495  maxlinkdim=16 maxerr=2.51E-16 time=0.008


MPS
[1] ((dim=2|id=701|"S=1/2,Site,n=1"), (dim=1|id=482|"Link,l=1"))
[2] ((dim=1|id=482|"Link,l=1"), (dim=2|id=83|"S=1/2,Site,n=2"), (dim=1|id=45|"Link,l=2"))
[3] ((dim=1|id=45|"Link,l=2"), (dim=2|id=843|"S=1/2,Site,n=3"), (dim=1|id=57|"Link,l=3"))
[4] ((dim=1|id=57|"Link,l=3"), (dim=2|id=342|"S=1/2,Site,n=4"), (dim=1|id=111|"Link,l=4"))
[5] ((dim=1|id=111|"Link,l=4"), (dim=2|id=217|"S=1/2,Site,n=5"), (dim=1|id=236|"Link,l=5"))
[6] ((dim=1|id=236|"Link,l=5"), (dim=2|id=982|"S=1/2,Site,n=6"), (dim=1|id=626|"Link,l=6"))
[7] ((dim=1|id=626|"Link,l=6"), (dim=2|id=85|"S=1/2,Site,n=7"), (dim=1|id=41|"Link,l=7"))
[8] ((dim=1|id=41|"Link,l=7"), (dim=2|id=852|"S=1/2,Site,n=8"))


In [77]:
function generate_mpds(psi)
    N = length(psi)
    psi = copy(psi)
    mpds = Dict{Int, ITensor}()
    
    for n = 1
        i = inds(psi[n], "Site")[1]
        j = commoninds(psi[n], psi[n+1])[1]
        
        Array_tensor1 = Array(psi[n], i,j)

        T = zeros(2,2,2,2)
        T[:,:,1,1] = Array_tensor1

        i2,j2,k2,l2 = prime(siteinds(psi, n)), prime(siteinds(psi, n+1)), siteinds(psi, n), siteinds(psi, n+1)
        tensor = ITensor(T, i2,j2,k2,l2)
        U,S,V = svd(tensor, i2,j2)

        S1 = delta(commonind(U,S), commonind(S,V))
        
        mpds[n] = U * S1 * V  
    end

    for n in 2:N-1
        i = inds(psi[n], "Site")[1]
        j = commoninds(psi[n], psi[n+1])[1]
        k = commoninds(psi[n-1], psi[n])[1]

        Array_tensor2 = Array(psi[n], i,j,k)

        G = zeros(2,2,2,2)
        G[:,:,:,1] = Array_tensor2 

        i2,j2,k2,l2 = prime(siteinds(psi, n)), prime(siteinds(psi, n+1)), siteinds(psi, n), siteinds(psi, n+1)
        tensor = ITensor(G, i2,j2,k2,l2)

        U,S,V = svd(tensor, i2,j2)
        S1 = delta(commonind(U,S), commonind(S,V))
        
        mpds[n] = U * S1 * V
    end

    for n = N
        i = inds((psi[n]), "Site")[1]
        j = commoninds(psi[n-1], psi[n])[1]
        mpds[n] = replaceinds(psi[n], i => prime(i), j => inds((psi[n]), "Site")[1])
    end
    return mpds
end

#This function helps with turning U into dag(U), and reverses the order such that we can apply to the MPS and disentangle it

function dagger!(mpds)
    mpds_dag = [dag(t) for t in mpds]
    for t in mpds_dag
        swapprime!(t, 0=>1)
    end
    return reverse(mpds_dag)
end

# Generate a circuit of depth D, with corresponding MPD gates.

function deep!(psi_non_truncated, D)
    N = length(sites)
    circuit_layers = Vector{Vector{ITensor}}(undef, D)
    
    psi_current = copy(psi_non_truncated)
    for d in 1:D
        psi_trunc = truncate(psi_current, maxdim=2)
        mpds = generate_mpds(psi_trunc)
        ordered_keys = sort(collect(keys(mpds)))
        mpds_vec = [mpds[n] for n in ordered_keys]
        mpds_vec = dagger!(mpds_vec)

        
        circuit_layers[d] = mpds_vec
        # Apply the MPDs to psi_current to get the next state
        psi_current = apply(mpds_vec, psi_current)
    end
    
    return circuit_layers
end


function reverse_deep!(circuit_layers::Vector{Vector{ITensor}})
    reversed_circuit = Vector{Vector{ITensor}}(undef, length(circuit_layers))
    
    for (i, layer) in enumerate(reverse(circuit_layers))
        reversed_layer = Vector{ITensor}(undef, length(layer))
        for (j, tensor) in enumerate(reverse(layer))
            reversed_layer[j] = swapprime(dag(tensor), 0=>1)
        end
        reversed_circuit[i] = reversed_layer
    end
    
    return reversed_circuit
end

function reverse_attract!(reversed_circuit, product_state)
    D = length(reversed_circuit)
    state = copy(product_state)
    
    for d in 1:D
        state = apply(reversed_circuit[d], state)
    end

    return state
end
#Usage
psi_non_truncated = copy(psi_non_truncated)
product_state = copy(product_state )
D = 1
circuit_layers = deep!(psi_non_truncated, D)
actual_circuit = reverse_deep!(circuit_layers)
state = reverse_attract!(actual_circuit, product_state)
norm = abs(inner(psi_non_truncated, state))

0.922630951606968

In [78]:
using HDF5

function package!(actual_circuit)
    # actual_circuit is an array of layers, each layer being an array of gates.
    D = length(actual_circuit)
    packaged_layers = Vector{Any}(undef, D)
    
    for d in 1:D
        layer = actual_circuit[d]
        N = length(layer)
        matrix_gates = Vector{Any}(undef, N)
        
        # Process the first N-1 gates as two-qubit gates.
        for n in 1:N-1
            gate = layer[n]
            s1_out = inds(gate, "Site")[1]
            s2_out = inds(gate, "Site")[2]
            s1_in  = inds(gate, "Site")[3]
            s2_in  = inds(gate, "Site")[4]
            gate_array = Array(gate, (s1_out, s2_out, s1_in, s2_in))
            matrix_gates[n] = reshape(gate_array, (4,4))
        end
        
        # Process the final gate (assumed to be a one-qubit gate).
        gate = layer[N]
        s1_out = inds(gate, "Site")[1]
        s1_in  = inds(gate, "Site")[2]
        gate_array = Array(gate, (s1_out, s1_in))
        matrix_gates[N] = reshape(gate_array, (2,2))
        
        packaged_layers[d] = matrix_gates
    end
    
    return packaged_layers
end

function savepackage!(filename::String, packaged_layers::Vector{Any})
    # Open an HDF5 file in write mode.
    h5open(filename, "w") do file
        # Loop over layers.
        for (layer_idx, layer) in enumerate(packaged_layers)
            # Loop over the gates in each layer.
            for (gate_idx, gate) in enumerate(layer)
                key = "layer$(layer_idx)_gate$(gate_idx)"
                write(file, key, gate)
            end
        end
    end
end

# Example usage:
# Suppose `actual_circuit` is already defined and you obtain packaged_layers:
packaged_layers = package!(actual_circuit)

# Let N be the number of sites (you can get it from the length of a layer) and D the depth.
N = length(actual_circuit[1])  # Number of gates in one layer corresponds to N sites.
D = length(actual_circuit)     # Depth of the circuit.

# Save the gates to an HDF5 file with a filename that encodes N and D.
filename = "gates_N$(N)_D$(D).h5"
savepackage!(filename, packaged_layers)
println("Saved circuit for N = $N and depth D = $D to ", filename)

Saved circuit for N = 8 and depth D = 1 to gates_N8_D1.h5
